In [ ]:
!pip -q install torch transformers sentence-transformers faiss-cpu flask pyngrok pyyaml bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 16.3 MB/s eta 0:00:00


In [ ]:
import os, json, time, yaml, re
from flask import Flask, request, jsonify
from sentence_transformers import SentenceTransformer
import faiss, numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# --- Redactor: strip PII from any free text before the model ---
EMAIL = re.compile(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
PHONE = re.compile(r"\+?\d[\d\-\s]{7,}\d")
ADDR  = re.compile(r"\b\d{1,5}\s+\w+(\s\w+){1,3}\b")

def redact(text:str)->str:
    text = EMAIL.sub("[redacted-email]", text)
    text = PHONE.sub("[redacted-phone]", text)
    text = ADDR.sub("[redacted-address]", text)
    return text

def contains_pii(text:str)->bool:
    return bool(EMAIL.search(text) or PHONE.search(text) or ADDR.search(text))


In [ ]:
KB = [
    {
        "id":"doc01",
        "title":"User Registration & Accounts",
        "content":"""ShopLite offers both buyer and seller accounts, giving flexibility for customers who only wish to shop and for businesses that want to sell. Buyers register by providing an email, a password, and optional phone number. Email verification is required within 24 hours to activate the account. Passwords must meet strong requirements: at least twelve characters, one number, and one symbol. Multi-factor authentication (MFA) via authenticator apps is supported to reduce account takeover risks.

Sellers begin with a buyer account and then upgrade. This process requires submitting business details such as tax ID, return address, and payout bank information. Verification is normally completed in 2–3 business days. Once approved, sellers access the Seller Dashboard where they can list products, manage orders, and monitor payouts.

Users can reset their password through secure email links. Accounts are locked after multiple failed attempts, but unlock automatically after 15 minutes or with a password reset. Account deletion requests permanently erase personal data within 30 days, except for legally required records like invoices and completed orders. Export of personal data is available before deletion for transparency."""},
    {
        "id":"doc02",
        "title":"Product Search & Filters",
        "content":"""ShopLite’s search system is designed to help customers find products quickly and accurately. Users can search by keyword or apply structured filters such as category, brand, price range, average rating, and availability. Typeahead suggestions surface up to five SKUs, with a preference for items in stock. These suggestions are refreshed hourly through cached results, ensuring speed while maintaining accuracy.

Natural language queries such as “under $50” or “4-star and up” are automatically translated into structured filters. Sorting options allow customers to view results by relevance (default), lowest price, highest price, rating, or newest arrivals. Misspellings are corrected when confidence exceeds a threshold of 0.8, minimizing frustration for customers.

Restricted or adult items never appear in general search results, regardless of filters. Additional safety checks prevent harmful or inappropriate queries from returning results. The search pipeline prioritizes relevance while balancing performance, with p95 latency goals kept under 300ms. By combining structured filtering, semantic understanding, and fast caching, ShopLite’s search ensures users can easily find what they are looking for."""},
    {
        "id":"doc03",
        "title":"Semantic Search & Relevance",
        "content":"""Beyond simple keyword search, ShopLite employs semantic embeddings to provide more intelligent results. This means queries such as “cheap running shoes” will retrieve items related to athletic footwear, even if the exact phrase does not match. All retrieved results must correspond to valid in-catalog SKUs; validators prevent any off-catalog hallucinations.

A reranking step evaluates candidate results, scoring them based on overlap with product titles, attributes, and historical click-through data. Items most relevant to the user’s intent are surfaced at the top. This ensures not only semantic accuracy but also alignment with real customer preferences.

Caching plays a large role in performance. Around 70% of frequent queries are cached, with refresh cycles set to maintain accuracy. P95 latency for typeahead is targeted at under 300ms, making suggestions feel instantaneous.

If queries contain banned or inappropriate terms, the system responds with a safe refusal message and alternative category links. By blending embeddings, rerankers, and guardrails, ShopLite ensures search remains useful, relevant, and aligned with customer expectations while minimizing integration risks."""},
    {
        "id":"doc04",
        "title":"Cart & Checkout",
        "content":"""The ShopLite cart supports products from multiple sellers, allowing customers to shop across the marketplace seamlessly. Items can be added, removed, or updated in quantity at any time. Customers can also mark items as “save for later,” creating a lightweight wishlist.

At checkout, users confirm shipping address, select preferred shipping speed, and choose a payment method. Tax and shipping estimates are displayed upfront for transparency. Promotional codes can be applied either in the cart or during checkout. Some codes stack, while others are restricted to avoid double discounts; combinability rules are clearly marked.

The cart persists across sessions for logged-in users and for seven days for guests using cookies. If inventory changes between sessions, users are notified before checkout. Items in short supply are flagged with warnings. Cart abandonment is tracked, and users may receive reminders or discounts to complete their purchases.

This flow ensures that customers have a smooth and trustworthy path from product discovery to order confirmation while giving sellers a reliable system for managing transactions."""},
    {
        "id":"doc05",
        "title":"Payments & Security",
        "content":"""ShopLite supports multiple payment methods including major credit and debit cards, Apple Pay, Google Pay (where available), and ShopLite gift cards. Payments are routed through PCI-compliant gateways, ensuring that sensitive card data never touches ShopLite servers. All transactions are encrypted in transit and monitored for unusual activity.

Refunds are issued back to the original payment method. Partial refunds are supported, giving sellers flexibility in handling returns or partial shipments. For customers in the EU, PSD2 and Strong Customer Authentication (SCA) may require additional verification steps such as one-time passcodes.

Security is prioritized at every stage. Suspicious or high-value transactions may be flagged and held for manual review. Customers are informed promptly by email if additional verification is needed. Payment history is securely stored in compliance with financial regulations.

ShopLite also runs automated fraud detection checks, which analyze patterns such as repeated failed payments, mismatched addresses, or unusually large orders. By combining modern payment convenience with strict security controls, the platform ensures safe and reliable transactions for both buyers and sellers."""},
    {
        "id":"doc06",
        "title":"Order Tracking & Delivery",
        "content":"""Once an order is placed, ShopLite assigns a unique order ID and provides a tracking link. Customers can follow the shipment status directly from their account dashboard or email confirmations. Common status updates include “Processing,” “Shipped,” “Out for delivery,” and “Delivered.”

Standard delivery typically ranges between 3–7 business days. Expedited shipping options are available depending on the carrier and destination. Shipping costs and estimated arrival dates are calculated during checkout so customers can make informed choices.

Customers can request an address change only before the order has been marked as “Shipped.” After that point, changes require carrier intervention and may delay delivery. If a delivery issue occurs, customers can initiate a support ticket. ShopLite coordinates with carriers to investigate lost, delayed, or damaged packages.

Delivery reliability is tracked across carriers to improve service quality. Notifications are sent by email or mobile app whenever status changes occur. With clear tracking and responsive support, ShopLite ensures customers remain confident about their order’s journey."""},
    {
        "id":"doc07",
        "title":"Returns & Refunds",
        "content":"""ShopLite offers a straightforward 30-day return policy, beginning from the date of delivery. Items must be unused and returned in their original packaging unless defective. Categories such as perishable goods, personalized items, or digital downloads are non-returnable, and these exceptions are clearly labeled on product pages.

Customers initiate returns from the Order Details page, where they can request a Return Merchandise Authorization (RMA). A prepaid shipping label may be provided depending on the seller’s policy. Once the return is received and inspected, refunds are issued within 5–10 business days to the original payment method.

If products arrive damaged or defective, customers may request replacements instead of refunds. Refunds include the product cost and applicable taxes, but shipping fees may be excluded unless the return is due to seller error.

ShopLite’s system keeps customers informed with return status updates by email and dashboard notifications. This clear and consistent approach to returns builds trust and reduces support overhead, ensuring customers feel secure in their purchases."""},
    {
        "id":"doc08",
        "title":"Seller Setup & Management",
        "content":"""To begin selling on ShopLite, individuals or businesses must upgrade their buyer account to a seller account. This process requires submitting verification documents such as tax ID, business address, and bank details for payouts. Verification typically takes 2–3 business days. Once approved, sellers gain access to the Seller Dashboard.

The dashboard enables storefront customization, including branding, return address, and shipping templates. Sellers can add products manually, upload CSV files, or use ShopLite’s API for large catalogs. Inventory and pricing can be updated in bulk.

Performance metrics such as on-time shipping, cancellation rate, and average rating are tracked and displayed on the dashboard. Consistently poor performance may result in penalties, warnings, or suspension of listings.

Payouts are issued weekly and include a detailed breakdown of sales, fees, and commissions. Sellers also have access to support resources and documentation. By offering robust tools and clear rules, ShopLite ensures that sellers can operate effectively while maintaining high standards for buyers."""},
    {
        "id":"doc09",
        "title":"Inventory Management",
        "content":"""ShopLite provides comprehensive tools for managing product inventory. Each SKU includes structured fields such as title, description, category, price, stock quantity, images, and shipping class. This ensures consistency across the marketplace.

Sellers can upload or edit products individually, through CSV bulk uploads, or via the ShopLite API. Inventory updates are reflected in near real-time. Low-stock alerts can be configured, notifying sellers when stock falls below a set threshold.

Backorders are allowed only for SKUs marked as “backorderable.” In these cases, the product detail page displays an estimated ship date, helping customers make informed decisions. Bulk editing features allow sellers to adjust pricing, availability, or categories for multiple SKUs simultaneously.

All changes are logged in the seller dashboard for accountability. Reports help sellers analyze sales trends, stock levels, and forecast demand. These features give sellers control over their inventory while ensuring that buyers always see accurate and up-to-date information when shopping."""},
    {
        "id":"doc10",
        "title":"Commission & Fees",
        "content":"""ShopLite earns revenue through commissions and fees. Each product category carries a base commission rate ranging from 5% to 15%. In addition, a fixed fee of $0.30 is charged per order item. This structure ensures that both low-cost and high-value items contribute fairly.

Refunds automatically adjust commissions. If an item is refunded, ShopLite returns the proportional commission to the seller. Additional services such as promoted listings, fulfillment, or premium storefront customization carry separate fees. These are clearly listed in the seller’s monthly statement.

Sellers receive a detailed invoice each month outlining all commissions, service fees, and adjustments. Transparency in fees helps sellers understand their costs and plan accordingly.

By balancing fair commission rates with optional paid features, ShopLite creates a sustainable business model that supports platform operations while rewarding seller success. The system ensures that sellers always know where their earnings are going."""},
    {
        "id":"doc11",
        "title":"Customer Support",
        "content":"""ShopLite provides multiple support channels to assist customers. These include a self-service help center, email support, and live chat. The help center prioritizes answers drawn directly from ShopLite’s policies and FAQ documents.

For order-related issues, live chat integrates with the order-status API. Customers can check their order progress in real time without waiting for an agent. If the query is outside scope, the chat system politely refuses and directs the customer to available resources.

Escalations from chat or email generate tickets in the support system. Standard response time for normal cases is within 24 hours. Urgent issues such as delivery failures or payment errors are flagged for resolution within 8 hours.

Support staff have access to detailed customer and order histories, enabling efficient troubleshooting. By combining automated self-service with human agents, ShopLite ensures that customer needs are addressed quickly, consistently, and fairly."""},
    {
        "id":"doc12",
        "title":"Reviews & Ratings",
        "content":"""ShopLite allows verified purchasers to leave product reviews and ratings. Ratings are provided on a 1–5 star scale, accompanied by optional text reviews. Offensive or inappropriate language is automatically filtered before posting.

Sellers may respond publicly to reviews once, allowing them to address customer concerns or thank buyers for feedback. This builds trust and transparency between buyers and sellers.

On product detail pages, reviews are summarized. Summaries must be based only on real phrases extracted from actual reviews. Automated systems prevent hallucinated claims or misleading statements.

Review authenticity is enforced by linking reviews to confirmed purchases. Customers cannot review products they have not purchased. Reports of fraudulent reviews are investigated, and confirmed violations may result in removal or penalties.

By encouraging honest feedback and providing clear safeguards, ShopLite maintains review quality and helps customers make confident purchasing decisions."""},
    {
        "id":"doc13",
        "title":"Mobile App",
        "content":"""The ShopLite mobile app mirrors the core features of the website, giving customers flexibility to shop on the go. Features include product search, filters, cart management, checkout, order tracking, and customer support chat.

Push notifications keep users informed of key events such as price drops, shipping updates, and return approvals. Customers can customize notification settings to avoid overload. Biometric login methods, including fingerprint and facial recognition, offer secure and convenient authentication.

An offline mode caches recently viewed pages and saved carts. While browsing is possible offline, checkout is disabled until the user reconnects. App updates are released regularly to maintain performance and security.

The app also integrates with device-level payment systems such as Apple Pay and Google Pay, making mobile checkout faster. By combining convenience, security, and continuity with the desktop site, the mobile app enhances the overall ShopLite shopping experience."""},
    {
        "id":"doc14",
        "title":"API for Developers",
        "content":"""ShopLite provides public APIs to allow developers to build integrations and apps. Access requires API keys, which can be generated in the developer portal. Standard rate limits are 60 requests per minute, with burst capacity up to 120 requests for short periods.

Key endpoints include product search, order tracking by order ID, and account management. Sensitive information such as personal data is excluded from responses to ensure privacy. Webhooks are available to notify developers when events occur, such as order status changes or refund approvals.

Errors follow the RFC 7807 format, returning a JSON object with fields for type, title, and detail. This standardized error handling makes integration easier and more predictable.

Developers are encouraged to review ShopLite’s API documentation for best practices. Abuse or violation of terms, such as scraping or excessive requests, may result in suspension of keys.

The API ecosystem ensures that ShopLite remains extendable, enabling external tools and services to integrate smoothly with the platform."""},
    {
        "id":"doc15",
        "title":"Security & Privacy",
        "content":"""Security and privacy are central to ShopLite’s operations. Personal data is minimized, encrypted at rest, and transmitted securely. Free-text inputs such as search queries or reviews pass through a redactor, which strips emails, phone numbers, and addresses before any AI processing.

Validation layers enforce strict scope for AI responses. Assistants must cite retrieved documents and refuse to answer if context is missing. This prevents unsupported claims and ensures accuracy.

ShopLite complies with data protection laws including GDPR and CCPA. Users can request a copy of their stored data and may delete their account entirely. Account deletion removes all personal information within 30 days, except for legally required records such as invoices.

In the event of a breach, ShopLite follows established legal timelines for disclosure and remediation. Regular penetration testing and monitoring are performed to detect vulnerabilities. Employees undergo training on secure data handling.

By combining proactive safeguards, regulatory compliance, and technical defenses, ShopLite ensures customer trust while continuing to innovate responsibly."""}
]

KB_TITLES = [d["title"] for d in KB]


In [ ]:
print(len(KB), "documents loaded")
print(KB_TITLES[:5])


15 documents loaded
['User Registration & Accounts', 'Product Search & Filters', 'Semantic Search & Relevance', 'Cart & Checkout', 'Payments & Security']


In [ ]:
import yaml

YAML_TEXT = r"""
version: "1.0"
created: "2025-09-30"
author: "Hiba Naim"

base_retrieval_prompt:
  role: |
    You are a helpful ShopLite customer assistant.
  goal: |
    Answer using only the provided document snippets.
  context_guidelines:
    - "Use only information from the provided snippets (no outside knowledge)"
    - "Quote key phrases when helpful and cite doc titles"
    - "If context is insufficient, refuse and suggest next steps"
  response_format: |
    Answer: <concise answer based strictly on context>
    Sources: <doc titles used>

multi_doc_synthesis:
  role: |
    You synthesize answers across multiple ShopLite docs.
  goal: |
    Combine policy + procedure correctly when the question spans topics.
  context_guidelines:
    - "Merge facts; avoid contradictions"
    - "Call out conditions and exceptions"
    - "List each source you used"
  response_format: |
    Answer: <step-by-step or bullet summary>
    Sources: <Doc A; Doc B>

no_context_refusal:
  role: |
    You are strict about scope.
  goal: |
    If no relevant snippets are retrieved, refuse to answer.
  context_guidelines:
    - "Say you don't have enough information"
    - "Suggest rephrasing or help center"
  response_format: |
    Answer: I don’t have enough ShopLite documentation to answer that.
    Next: Please rephrase or visit the Help Center.
    Sources: none

clarification_needed:
  role: |
    You request precise details before answering.
  goal: |
    Ask for 1–2 concrete clarifying questions when the query is ambiguous.
  context_guidelines:
    - "Ask short questions"
    - "Offer likely options"
  response_format: |
    Clarify: <two short questions to narrow scope>
    Example options: <bulleted options>
    Sources: pending

safety_guardrails:
  role: |
    You enforce PII redaction and output validation.
  goal: |
    Ensure answers never reveal PII and remain within provided docs.
  context_guidelines:
    - "Do not echo emails/phones/addresses"
    - "Do not fabricate SKUs or policies"
  response_format: |
    Answer: <safe, grounded reply>
    Policy: <which rule constrained the answer>
    Sources: <docs>
"""

PROMPTS = yaml.safe_load(YAML_TEXT)
print(list(PROMPTS.keys()))


['version', 'created', 'author', 'base_retrieval_prompt', 'multi_doc_synthesis', 'no_context_refusal', 'clarification_needed', 'safety_guardrails']


In [ ]:
emb_model = SentenceTransformer("all-MiniLM-L6-v2")
doc_vecs = emb_model.encode([d["content"] for d in KB], convert_to_numpy=True, normalize_embeddings=True)

index = faiss.IndexFlatIP(doc_vecs.shape[1])
index.add(doc_vecs)

def retrieve(query:str, k:int=5):
    qv = emb_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    scores, idxs = index.search(qv, k)
    results = []
    for i, s in zip(idxs[0], scores[0]):
        d = KB[int(i)]
        results.append({"title": d["title"], "content": d["content"], "score": float(s)})
    return results


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Cell 5: LLM loading and setup (distilgpt2)
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

MODEL_ID = "distilgpt2"  # tiny + open-source, perfect for demo

print("Loading model:", MODEL_ID)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# DistilGPT2 sometimes doesn’t define a pad token → use EOS instead
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

# Load model
model = AutoModelForCausalLM.from_pretrained(MODEL_ID)

# Setup pipeline (short outputs to keep fast)
gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",     # GPU if available
    max_new_tokens=96,     # short outputs for speed
    do_sample=False,       # greedy decoding
    top_p=1.0,
    pad_token_id=tokenizer.pad_token_id,
)


print("Model ready ✅")


Loading model: distilgpt2


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Model ready ✅


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 1) Create embeddings model for retrieval
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Prepare FAISS index
doc_embeddings = embedder.encode([d["content"] for d in KB])
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(np.array(doc_embeddings))

print("FAISS index built ✅")

# 2) Retrieval function
def retrieve(query, k=2):
    query_emb = embedder.encode([query])
    D, I = index.search(np.array(query_emb), k)
    return [KB[i] for i in I[0]]

# 3) Generate answer using retrieved docs + distilgpt2
def rag_answer(query, k=2):
    docs = retrieve(query, k)
    context = "\n\n".join([f"{d['title']}: {d['content']}" for d in docs])
    sources = "; ".join([d["title"] for d in docs])

    prompt = f"""You are a ShopLite assistant.
Answer the user using only the context below.
If the answer is not in the context, reply exactly: "I don’t know based on ShopLite docs."

Context:
{context}

Question: {query}
Answer:"""

    output = gen(prompt, max_new_tokens=128)[0]["generated_text"]

    # Post-process: cut answer after "Answer:" to remove prompt echo
    answer = output.split("Answer:")[-1].strip()

    # Guardrail: if model ignored instructions and rambled nonsense
    if not any(keyword in answer.lower() for keyword in ["shoplite", "seller", "buyer", "account", "verification", "dashboard", "tax id"]):
        answer = "I don’t know based on ShopLite docs."

    return {"answer": answer, "sources": sources}


FAISS index built ✅


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/health", methods=["GET"])
def health():
    return jsonify({"status": "ok", "message": "RAG system is running"}), 200

@app.route("/ping", methods=["POST"])
def ping():
    data = request.get_json()
    text = data.get("text", "")
    if not text:
        return jsonify({"error": "No text provided"}), 400
    out = gen(text, max_new_tokens=60)[0]["generated_text"]
    return jsonify({"response": out})

@app.route("/chat", methods=["POST"])
def chat():
    data = request.get_json()
    query = data.get("query", "")
    if not query:
        return jsonify({"error": "No query provided"}), 400

    try:
        answer = rag_answer(query)
        return jsonify({
            "query": query,
            "answer": answer
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

  # === NEW Week-5 endpoint ===
@app.route("/generate", methods=["POST"])
def generate_simple():
    """
    Simple LLM generation endpoint for Week-5 backend.
    No retrieval — just direct prompt completion.
    """
    data = request.get_json(force=True)
    prompt = data.get("prompt", "")
    max_tokens = int(data.get("max_tokens", 200))

    if not prompt:
        return jsonify({"error": "Missing prompt"}), 400

    try:
        out = gen(
            prompt,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=0.2,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )[0]["generated_text"]

        return jsonify({"text": out})
    except Exception as e:
        return jsonify({"error": str(e)}), 500



In [ ]:
from pyngrok import ngrok

# Ask for ngrok token securely
NGROK_TOKEN = input("Enter your ngrok token: ")

# Auth + start tunnel
ngrok.set_auth_token(NGROK_TOKEN)
public_url = ngrok.connect(5000)

print("✅ ngrok tunnel created:")
print("Public URL:", public_url)

# Start Flask app
import threading

def run_app():
    app.run(port=5000)

# Run Flask in background thread
threading.Thread(target=run_app).start()



Enter your ngrok token: 33V6npEpfB8PNLGzXL1glRcUXdX_C68z5eUkEzsHpRFL72St
✅ ngrok tunnel created:
Public URL: NgrokTunnel: "https://multivocal-undulatingly-tatum.ngrok-free.dev" -> "http://localhost:5000"


In [ ]:
import requests

BASE = "https://multivocal-undulatingly-tatum.ngrok-free.dev"

# Health check
print("Health:", requests.get(f"{BASE}/health").json())

# Ping test
print("Ping:", requests.post(f"{BASE}/ping", json={"text": "Hello!"}).json())

# Chat (RAG) test
q = "How do I register as a seller on ShopLite?"
res = requests.post(f"{BASE}/chat", json={"query": q}).json()
print("Chat response:", res)


INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 11:33:31] "GET /health HTTP/1.1" 200 -


Health: {'message': 'RAG system is running', 'status': 'ok'}


INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 11:33:33] "POST /ping HTTP/1.1" 200 -


Ping: {'response': 'Hello!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'}


INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 11:33:42] "POST /chat HTTP/1.1" 200 -


Chat response: {'answer': {'answer': 'The seller is required to provide a valid account. The buyer agrees to provide a valid refund. The seller must also provide the full description of the buyer that he or she has purchased.\nAsk a customer if they want to purchase or sell the item.\nIf the buyer is a seller, if they only want to purchase it, the buyer will have the option of having the option to apply for the refund.\nAsk a seller if they want to purchase or sell the item.\nAsk a seller for the item.\nIf you already do, the seller will have the choice of purchasing the item.\nAsk the seller if they', 'sources': 'Seller Setup & Management; User Registration & Accounts'}, 'query': 'How do I register as a seller on ShopLite?'}


In [ ]:
import requests, time

# wait for Flask
time.sleep(2)

BASE = public_url.public_url  # <-- get just the https://... string
print("Public URL:", BASE)

print("Health:", requests.get(f"{BASE}/health").json())
print("Ping:", requests.post(f"{BASE}/ping", json={"text": "Hello!"}).json())

q = "How do I register as a seller on ShopLite?"
res = requests.post(f"{BASE}/chat", json={"query": q}).json()
print("Chat:", res)


Public URL: https://multivocal-undulatingly-tatum.ngrok-free.dev


INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 11:33:49] "GET /health HTTP/1.1" 200 -


Health: {'message': 'RAG system is running', 'status': 'ok'}


INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 11:33:52] "POST /ping HTTP/1.1" 200 -


Ping: {'response': 'Hello!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'}


INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 11:33:59] "POST /chat HTTP/1.1" 200 -


Chat: {'answer': {'answer': 'Customers can register as a seller by submitting a short description or a description.\nUser Registration & Accounts: ShopLite offers a short description or a description.\nUser Registration & Accounts: ShopLite offers a short description or a description.\nUser Registration & Accounts: ShopLite provides a short description or a description.\nUser Registration & Accounts: ShopLite provides a short description or a description.\nDescription: ShopLite provides a short description or a description.\nUser Registration & Accounts: ShopLite provides a short description or a description.\nUser Registration & Accounts', 'sources': 'Seller Setup & Management; User Registration & Accounts'}, 'query': 'How do I register as a seller on ShopLite?'}


In [ ]:
import requests

BASE = public_url.public_url   # just the https://... string from ngrok
print("Health:", requests.get(f"{BASE}/health").json())


INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 11:34:09] "GET /health HTTP/1.1" 200 -


Health: {'message': 'RAG system is running', 'status': 'ok'}


In [ ]:
print("Ping:", requests.post(f"{BASE}/ping", json={"text": "Hello!"}).json())


INFO:werkzeug:127.0.0.1 - - [23/Oct/2025 11:34:14] "POST /ping HTTP/1.1" 200 -


Ping: {'response': "Hello! I hope that you enjoy reading this and please remember to check out the article to see why this is an issue and why it is not a problem.\n\n\nHow do you explain the problem?\nThe problem is the same as when you look at the problem. If you don't know why"}
